In [1]:
import pandas as pd
import numpy as np

# Tree computation

## Binomial

In [2]:
def node_bin(S0,u,d,additive=True,dividend=0,N=1):
    if additive==True:
        S=(S0-dividend) +(d * (np.arange(N,-1,-1)) + u * (np.arange(0,N+1,1)) )
    elif additive==False:
        S=(S0-dividend) *d ** (np.arange(N,-1,-1)) * u ** (np.arange(0,N+1,1)) 
    return S

In [3]:
def binomial(S0, u,d, additive=True,dividend=0,N=1):
    l={}
    for i in range(0,N+1):
        l[i]=node_bin(S0,u,d,N=i,additive=additive,dividend=dividend)
    return l

In [11]:
binomial(2,0.08,-0.03,N=2)

{0: array([2.]), 1: array([1.97, 2.08]), 2: array([1.94, 2.05, 2.16])}

## Trinomial

In [4]:
def node_trin(S0,u,m,d,additive=True,dividend=0):
    if additive==True:
        S=S0+np.eye(3,3)@(np.array([d,m,u]))
    elif additive==False:
        S=S0*np.eye(3,3)@(np.array([d,m,u]))
    return S

In [13]:
def trinominal(S0,u,m,d,additive=True,dividend=0,N=1):
    a={}
    for c in range(0,N):
        l=[]
        if c==0:
            n=S0
            l=node_trin(n,u,m,d,additive,dividend)
        else:
            n=a[c-1]
            for i in range(0,len(n)):
                l.append(node_trin(n[i],u,m,d,additive,dividend))
            l=np.concatenate(l)
        a[c]=l
    return a

In [6]:
u,m,d=1.23,0.7,0.6

tr=trinominal(1,u,m,d,additive=False,N=2)

# Absence Of Arbitrage

In [7]:
def aoa(n,s0,r):
    if (n[0]<s0*(1+r))and (n[1]>s0*(1+r)):
        print('Ok')
    else:
        print('Arbitrage Opportunity!')
        if n[0]>=s0*(1+r):
            print('Check down')
        if n[1]<=s0*(1+r):
            print('Check up')

In [14]:
aoa(n=[0.0162,0.01836],s0=0.018,r=0.015)

Ok


# Probability measures 

In [15]:
def solving_b(n,s0):
    u=(s0-n[0])/(n[1]-n[0])
    d=1-u
    return d,u

In [23]:
from sympy import solve
from sympy import symbols
import sympy
import re

def solving_t(n,s0):

    x1,x2,x3 = symbols('x_d x_m x_u')
    aa=[]
    for i in [(x1,x2),(x2,x3),(x3,x1)]:
        l=[]
        eq=[(x1+x2+x3-1),(n[0]*x1+n[1]*x2+n[2]*x3-s0)]
        solution=sympy.solve(eq,i)
        print(solution)
        for x in [0,1]:
            for a in [0,1]:
                if str(list(solution.values())[x])[0]=='-':
                    print(str(list(solution.values())[x])[0])
                    l.append(float(re.findall('[-+]*.\d.\d+',str(list(solution.values())[x]))[a].replace(" ","")))

                else:
                    l.append(float(re.findall('[-+]*.\d.\d+','+'+str(list(solution.values())[x]))[a].replace(" ","")))
        aa.append(np.array(l))

    return prob(aa)

def prob(aa):
    xx=np.zeros((3,2))
    ## DOWN
    xx[0][1]=-aa[1][0]/aa[1][1]      # +
    xx[0][0]=-(aa[1][0]-1)/aa[1][1]  # -

    ## MIDDLE
    xx[1][1]=-aa[2][2]/aa[2][3]      # +
    xx[1][0]=-(aa[2][0]-1)/aa[2][1] # -

    ## UP
    xx[2][1]=-aa[0][2]/aa[0][3]
    xx[2][0]=-aa[0][1]/aa[0][0]

    return pd.DataFrame(xx.clip(0),index=['Down','Middle','Up'],columns=['Min','Max'])

In [55]:
solving_t([0.6,0.7,1.23], 1/0.9)

{x_d: 5.3*x_u - 4.11111111111111, x_m: 5.11111111111111 - 6.3*x_u}
{x_m: 0.224318658280922 - 1.18867924528302*x_d, x_u: 0.188679245283019*x_d + 0.775681341719078}
{x_u: 0.811287477954145 - 0.158730158730159*x_m, x_d: 0.188712522045855 - 0.841269841269841*x_m}


,Min,Max
Down,0.000000,0.188713
Middle,0.000000,0.224319
Up,0.775681,0.811287


In [87]:
trinominal(1,u=1.23,m=0.7,d=0.6,additive=False,N=2)[1]**2

array([0.1296    , 0.1764    , 0.544644  , 0.1764    , 0.2401    ,
       0.741321  , 0.544644  , 0.741321  , 2.28886641])

In [93]:
0.622*0.23+0.96*0.77

0.8822599999999999

In [95]:
solving_t([1.06,1.25,2.75],2.08)

{x_d: 7.89473684210526*x_u - 4.36842105263158, x_m: 5.36842105263158 - 8.89473684210526*x_u}
{x_m: 0.446666666666667 - 1.12666666666667*x_d, x_u: 0.126666666666667*x_d + 0.553333333333333}
{x_u: 0.603550295857988 - 0.112426035502959*x_m, x_d: 0.396449704142012 - 0.887573964497041*x_m}


,Min,Max
Down,0.000000,0.396450
Middle,0.000000,0.446667
Up,0.553333,0.603550


In [37]:
trinominal(1,u=1.23,m=0.7,d=0.6, additive=False, N=2)

{0: array([0.6 , 0.7 , 1.23]),
 1: array([0.36  , 0.42  , 0.738 , 0.42  , 0.49  , 0.861 , 0.738 , 0.861 ,
        1.5129])}

In [140]:
solving(tr[1][0:3],tr[0][0]/0.91)

{x_d: 5.3*x_u - 3.98901098901099, x_m: 4.98901098901099 - 6.3*x_u}
{x_m: 0.247356417167738 - 1.18867924528302*x_d, x_u: 0.188679245283019*x_d + 0.752643582832262}
{x_u: 0.79190650619222 - 0.158730158730159*x_m, x_d: 0.20809349380778 - 0.841269841269841*x_m}


,Min,Max
Down,0.000000,0.208093
Middle,0.000000,0.247356
Up,0.752644,0.791907


In [141]:
solving(tr[1][3:6],tr[0][1]/0.91)

{x_d: 5.3*x_u - 3.98901098901099, x_m: 4.98901098901099 - 6.3*x_u}
{x_m: 0.247356417167738 - 1.18867924528302*x_d, x_u: 0.188679245283019*x_d + 0.752643582832262}
{x_u: 0.79190650619222 - 0.158730158730159*x_m, x_d: 0.20809349380778 - 0.841269841269841*x_m}


,Min,Max
Down,0.000000,0.208093
Middle,0.000000,0.247356
Up,0.752644,0.791907


In [135]:
solving(tr[1][6:9],0.91*tr[0][2])

{x_d: 5.3*x_u - 2.1, x_m: 3.1 - 6.3*x_u}
{x_m: 0.60377358490566 - 1.18867924528302*x_d, x_u: 0.188679245283019*x_d + 0.39622641509434}
{x_u: 0.492063492063492 - 0.158730158730159*x_m, x_d: 0.507936507936508 - 0.841269841269841*x_m}


,Min,Max
Down,0.000000,0.507937
Middle,0.000000,0.603774
Up,0.396226,0.492063


In [220]:
solving_t([1.502,1.603,3.017],1.3/0.783)

{x_d: 14.0*x_u - 0.567138323027723, x_m: 1.56713832302772 - 15.0*x_u}
{x_m: 0.959490119783734 - 1.07142857142857*x_d, x_u: 0.0714285714285714*x_d + 0.0405098802162659}
{x_u: 0.104475888201848 - 0.0666666666666667*x_m, x_d: 0.895524111798152 - 0.933333333333333*x_m}


,Min,Max
Down,0.00000,0.895524
Middle,0.00000,0.959490
Up,0.04051,0.000000


In [221]:
solving_t([1.402,1.503,2.917],1.2/0.783)

{x_d: 14.0*x_u - 0.292743067410198, x_m: 1.2927430674102 - 15.0*x_u}
{x_m: 0.979089780899272 - 1.07142857142857*x_d, x_u: 0.0714285714285714*x_d + 0.0209102191007284}
{x_u: 0.0861828711606799 - 0.0666666666666667*x_m, x_d: 0.91381712883932 - 0.933333333333333*x_m}


,Min,Max
Down,0.00000,0.913817
Middle,0.00000,0.979090
Up,0.02091,0.000000


In [224]:
solving_t([0.55,1.1,3.6],1.0989)

{x_d: 4.54545454545455*x_u + 0.002, x_m: 0.998 - 5.54545454545455*x_u}
{x_m: 1.00044 - 1.22*x_d, x_u: 0.22*x_d - 0.00044}
{x_u: 0.179967213114754 - 0.180327868852459*x_m, x_d: 0.820032786885246 - 0.819672131147541*x_m}


,Min,Max
Down,0.000361,0.820033
Middle,0.000000,1.000440
Up,0.000000,0.179967


In [227]:
1-1.22*0.41

0.4998

In [232]:
0.22*0.41

0.09019999999999999